In [1]:
# !pip install -U git+https://github.com/openfisca/openfisca-ceq.git@master#egg=OpenFisca-CEQ
# !pip install -U git+https://github.com/openfisca/openfisca-senegal.git@ceq#egg=OpenFisca-Senegal
# !pip install -U git+https://github.com/openfisca/openfisca-mali.git@master#egg=OpenFisca-Mali
# !pip install -U git+https://github.com/openfisca/openfisca-cote-d-ivoire.git@master#egg=OpenFisca-COTE-D-IVOIRE

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data.tax_targets import read_tax_target
from openfisca_ceq.tools.results.inequality import inequality_table
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )

In [3]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable['autres_revenus_du_capital_brut'] = 'autres_revenus_du_capital_brut'
ceq_by_harmonized_variable['pension_retraite_brut'] = 'pension_retraite_brut'
ceq_by_harmonized_variable['revenu_foncier_brut'] = 'revenu_foncier_brut'
ceq_by_harmonized_variable['revenu_non_salarie_brut'] = 'revenu_non_salarie_brut' 
ceq_by_harmonized_variable['salaire_brut'] = 'salaire_brut'
ceq_by_harmonized_variable['salaire_super_brut'] = 'salaire_super_brut'


## Inequality tables

In [4]:
for country, year in year_by_country.items():
    survey_scenario = build_ceq_survey_scenario(country, year)
    inflated_survey_scenario = build_ceq_survey_scenario(country, year, inflate = True)    
    data = [
        (
            harmonized_variable, 
            openfisca_variable, 
            survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            inflated_survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            ) 
        for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
        ]
    revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate", "inflated aggregate"])
    display(revenus.round(0).astype({"aggregate": int, "inflated aggregate": int}))
    display(country)
    display(inequality_table(survey_scenario))
    display(country + " (inflated)")
    display(inequality_table(survey_scenario))

/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is no

,harmonized,openfisca,aggregate,inflated aggregate
0,rev_i_autoconsommation,autoconsumption,1288,1952
1,rev_i_autres,other_income,580,878
2,rev_i_autres_transferts,gifts_sales_durables,433,656
3,rev_i_loyers_imputes,imputed_rent,277,420
4,rev_i_transferts_publics,direct_transfers,8,8
5,rev_i_agricoles,revenu_agricole,1426,2161
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,75,75
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,1346,2039
8,rev_i_independants_taxe,revenu_non_salarie,244,244
9,rev_i_independants,revenu_non_salarie_total,1590,2283


'cote_d_ivoire'

,market_income,market_income_plus_pensions,gross_income,disposable_income,consumable_income,final_income
Gini,0.60,0.59,0.59,0.58,0.62,0.58
Bottom 40 %,0.08,0.08,0.08,0.08,0.07,0.09
Top 10 %,0.47,0.47,0.47,0.46,0.48,0.45


'cote_d_ivoire (inflated)'

...

,harmonized,openfisca,aggregate,inflated aggregate
0,rev_i_autoconsommation,autoconsumption,157,239
1,rev_i_autres,other_income,148,225
2,rev_i_autres_transferts,gifts_sales_durables,644,981
3,rev_i_loyers_imputes,imputed_rent,327,497
4,rev_i_transferts_publics,direct_transfers,10,10
5,rev_i_agricoles,revenu_agricole,937,1427
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,13,13
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,828,1261
8,rev_i_independants_taxe,revenu_non_salarie,264,264
9,rev_i_independants,revenu_non_salarie_total,1092,1525


'senegal'

,market_income,market_income_plus_pensions,gross_income,disposable_income,consumable_income,final_income
Gini,0.53,0.52,0.52,0.51,0.53,0.50
Bottom 40 %,0.11,0.11,0.11,0.11,0.10,0.12
Top 10 %,0.40,0.40,0.40,0.39,0.40,0.38


'senegal (inflated)'

,market_income,market_income_plus_pensions,gross_income,disposable_income,consumable_income,final_income
Gini,0.53,0.52,0.52,0.51,0.53,0.50
Bottom 40 %,0.11,0.11,0.11,0.11,0.10,0.12
Top 10 %,0.40,0.40,0.40,0.39,0.40,0.38


In [5]:
result = read_tax_target(add_country_details=True)
display(result)

/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is no

Cote d'Ivoire                  \
                                                       actual direct inflated   
Total tax revenue                                     2412.69    NaN      NaN   
Direct taxes                                              729  178.9    344.4   
Personal Income Taxes                                   334.7  178.9    344.4   
Payroll Tax and social contributions                        -    NaN      NaN   
Corporate Income Tax                                    319.8    0.0      0.0   
Other Direct Taxes                                       74.5    NaN      NaN   
Indirect taxes                                        1683.69  890.2    961.3   
VAT                                                     617.9  682.3    736.8   
Import Taxes                                          516.092  207.9    224.5   
Excise taxes                                             34.4    0.0      0.0   
on Oil Derivates                                            -    NaN      NaN   
on alcohol, tabac and other non-oil derivatives          34.4    NaN      NaN   
Other Indirect Taxes                                    515.3    NaN      NaN   
Share simulated taxes (% of total tax revenue)            NaN    NaN      NaN   
impot_general_revenu                                      NaN  155.8    309.4   
impot_revenu_creances                                     NaN   16.4     24.9   
impot_revenu_foncier                                      NaN    6.7     10.1   
impot_traitement_salaire                                  NaN    NaN      NaN   
contribution_globale_fonciere                             NaN    NaN      NaN   
contribution_globale_unique                               NaN    NaN      NaN   
droit_progressif_pension_retraite                         NaN    NaN      NaN   
droit_progressif_salaire                                  NaN    NaN      NaN   
droit_progressif                                          NaN    NaN      NaN   
droit_proportionnel_autres_revenus                        NaN    NaN      NaN   
droit_proportionnel_salaire                               NaN    NaN      NaN   
droit_proportionnel                                       NaN    NaN      NaN   
impot_revenus                                             NaN    NaN      NaN   

                                                  Mali                  \
                                                actual direct inflated   
Total tax revenue                                775.8    NaN      NaN   
Direct taxes                                     220.8    7.4     40.9   
Personal Income Taxes                             65.4    7.4     40.9   
Payroll Tax and social contributions                 -    NaN      NaN   
Corporate Income Tax                             113.3    0.0      0.0   
Other Direct Taxes                                42.1    NaN      NaN   
Indirect taxes                                     555  267.3    413.5   
VAT                                              282.2  235.4    364.1   
Import Taxes                                     112.1   31.9     49.3   
Excise taxes                                       4.7    0.0      0.0   
on Oil Derivates                                   4.7    NaN      NaN   
on alcohol, tabac and other non-oil derivatives      -    NaN      NaN   
Other Indirect Taxes                               156    NaN      NaN   
Share simulated taxes (% of total tax revenue)     NaN    NaN      NaN   
impot_general_revenu                               NaN    NaN      NaN   
impot_revenu_creances                              NaN    NaN      NaN   
impot_revenu_foncier                               NaN    NaN      NaN   
impot_traitement_salaire                           NaN    7.4     40.9   
contribution_globale_fonciere                      NaN    NaN      NaN   
contribution_globale_unique                        NaN    NaN      NaN   
droit_progressif_pension_retraite                  NaN    NaN      NaN